In [1]:
import osmnx as ox
import networkx as nx
import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import os
import pandas as pd
from shapely.ops import unary_union

In [2]:
# define the place name for the west midlands area
place_name = "West midlands, UK"
# download the road network data from osmnx from the OSM API
RoadNetworkGraph = ox.graph_from_place(place_name, network_type='drive')
print(f"Original CRS: {RoadNetworkGraph.graph['crs']}")
# project the road network to the British National Grid (EPSG:27700)
RoadNetworkGraph_projected = ox.project_graph(RoadNetworkGraph, to_crs='EPSG:27700')
# check the CRS
print(f"Projected CRS: {RoadNetworkGraph_projected.graph['crs']}")

Original CRS: epsg:4326
Projected CRS: EPSG:27700


In [3]:
# check how many nodes and edges in the graph
print(f"Number of nodes: {len(RoadNetworkGraph_projected.nodes)}")
print(f"Number of edges: {len(RoadNetworkGraph_projected.edges)}")

Number of nodes: 83323
Number of edges: 184282


In [4]:
# check if the network is a directed graph
print(f"Is the graph directed? {nx.is_directed(RoadNetworkGraph_projected)}")
# change the graph to an undirected graph
RoadNetworkGraph_undirected = RoadNetworkGraph_projected.to_undirected()
# check if the network is fully connected
print(f"Is the graph fully connected? {nx.is_connected(RoadNetworkGraph_undirected)}")

Is the graph directed? True
Is the graph fully connected? True


In [5]:
import pulp
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon
import numpy as np

In [6]:
# import the boundary of the west midlands area
Area_boundary = gpd.read_file(os.path.join("data/westmidslands/westmidlands.shp"))


In [7]:
# check area_boundary's crs
print(f"Area_boundary's CRS: {Area_boundary.crs}")

Area_boundary's CRS: EPSG:27700


In [8]:
# let us generate a grid of points within the boundary of the west midlands area
polygon = Area_boundary.geometry.iloc[0]

# define the grid spacing (1000 meters)
grid_spacing = 1000  # 1000 meters

# get the bounds of the polygon
minx, miny, maxx, maxy = polygon.bounds

# generate the x and y coordinates for the grid points
x_coords = np.arange(minx, maxx, grid_spacing)
y_coords = np.arange(miny, maxy, grid_spacing)
grid_points = [Point(x, y) for x in x_coords for y in y_coords]

# filter out the points that are outside the polygon
points_within_polygon = [point for point in grid_points if polygon.contains(point)]

# create a geodataframe for the points
Candidate_points_1000_gdf = gpd.GeoDataFrame(geometry=points_within_polygon, crs=Area_boundary.crs)


In [9]:
# let the index of the candidate points be the candidate point id
Candidate_points_1000_gdf['point_id'] = Candidate_points_1000_gdf.index
Candidate_points_1000_gdf.columns

Index(['geometry', 'point_id'], dtype='object')

In [ ]:
# calculate the euclidean distance between each candidate points and demands points

In [32]:
# import the demandpoints csv data
DemandPoints_raw = pd.read_csv("data/processed/mobilisation.csv")

In [34]:
DemandPoints_raw.columns

Index(['Incident_Number', 'callsign_type', 'call_time', 'callsign_station',
       'initial_incident_type', 'incident_classification_label',
       'incident_profile_label', 'incident_classification_level1',
       'call_seconds', 'reaction_seconds', 'driving_seconds',
       'on_scene_seconds', 'EASTINGS', 'NORTHINGS', 'call_hour', 'call_day',
       'call_month', 'call_year', 'Station name', 'Sta_Easting',
       'Sta_Northing', 'PRL_Count', 'BRV_Count', 'Closed (Y/N)', 'Opened',
       'Closed'],
      dtype='object')

In [36]:
# create a new column called 'id' for the demand points same as thier index
DemandPoints_raw['id'] = DemandPoints_raw.index

In [39]:
DemandPoints = DemandPoints_raw[DemandPoints_raw['call_year'] == 2023][['id','EASTINGS','NORTHINGS','incident_profile_label']].sample(1000, random_state=1)
DemandPoints.sample(5)

,id,EASTINGS,NORTHINGS,incident_profile_label
513123,513123,433570.980020,278379.786107,Low Risk
515872,515872,405163.291644,284321.225580,False Alarms
538103,538103,406656.299917,286446.205850,Low Risk
533388,533388,400925.163139,301258.756411,High Risk
527977,527977,429447.357819,277995.022672,False Alarms


In [ ]:

# visualize the grid points and the boundary of the west midlands area
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
Area_boundary.plot(ax=ax, edgecolor='black', facecolor='none')
points_gdf.plot(ax=ax, color='red', markersize=1)
plt.show()


In [ ]:

# 假设有4个需求点
demand_points = [1, 2, 3, 4]

# 定义距离矩阵
# 4个需求点到10个已有设施点的距离矩阵 (4x10)
distance_matrix_existing = np.array([
    [0, 2, 4, 7, 3, 8, 6, 4, 5, 9],
    [2, 0, 1, 5, 2, 3, 8, 7, 6, 2],
    [4, 1, 0, 3, 7, 5, 2, 6, 8, 4],
    [7, 5, 3, 0, 4, 7, 3, 2, 5, 6]
])

# 4个需求点到50个候选位置的距离矩阵 (4x50)
distance_matrix_candidates = np.random.randint(1, 10, size=(4, 50))

# 合并已有设施点和候选位置的距离矩阵 (4x60)
distance_matrix = np.hstack((distance_matrix_existing, distance_matrix_candidates))

# 现有的设施点位置
current_facilities = list(range(1, 11))  # 设施点1到10
# 需要重新选址的设施点数量
n_relocate = 2
# 候选位置（50个）
candidate_locations = list(range(11, 61))  # 设施点11到60

# 初始化优化问题
problem = pulp.LpProblem("P-Center_Problem_with_Relocation", pulp.LpMinimize)

# 定义决策变量
x = pulp.LpVariable.dicts("Facility", candidate_locations, 0, 1, pulp.LpBinary)
y = pulp.LpVariable("Max_Distance")

# 添加目标函数：最小化最大距离
problem += y

# 添加约束条件
for i in range(len(demand_points)):
    problem += y >= pulp.lpSum(distance_matrix[i][j-1] * x[j] for j in candidate_locations)

problem += pulp.lpSum(x[j] for j in candidate_locations) == n_relocate

# 确保现有的部分设施位置固定不变
for facility in current_facilities:
    problem += x[facility] == 1

# 求解问题
problem.solve()

# 输出结果
print("Status:", pulp.LpStatus[problem.status])
print("Max Distance:", pulp.value(y))
print("Selected Facilities:")
for j in candidate_locations:
    if pulp.value(x[j]) == 1:
        print(f"Facility at candidate location {j}")
